In [1]:
import sys

sys.path.insert(0, "..")

### Setting here


In [2]:
# Paths
OFFLINE_DATASET_DIR = "D:/storage/odl/cache/offline/reintel2020/dataset.pt"
CHECKPOINT_DIR = "D:/storage/odl/checkpoints/offline/reintel2020/TextCNN_BERT/"

# Hyperparameters for TextCNN with trained-WE
MAX_LENGTH = 65
EMBEDDING_DIM = 768
NUM_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
LR = 1e-5

### Load dataset from cache


In [3]:
import torch

dataset = torch.load(OFFLINE_DATASET_DIR)
train_dataset = [
    {k: v for k, v in d.items() if k in ["post_message", "label"]}
    for d in dataset["train"].data
]
test_dataset = [
    {k: v for k, v in d.items() if k in ["post_message", "label"]}
    for d in dataset["test"].data
]

del dataset

len(train_dataset), len(test_dataset)

(3497, 875)

### Define model


In [4]:
from module import TextCNN, BertEmbedding
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = BertEmbedding(max_length=MAX_LENGTH, device=device).to(device)
F = TextCNN(EMBEDDING_DIM, NUM_FILTERS, FILTER_SIZES).to(device)

optimizer = torch.optim.Adam(list(G.parameters()) + list(F.parameters()), lr=LR)
criterion = nn.CrossEntropyLoss()

G, F

(BertEmbedding(
   (bert): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(64001, 768, padding_idx=1)
       (position_embeddings): Embedding(258, 768, padding_idx=1)
       (token_type_embeddings): Embedding(1, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0-11): 12 x RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
              

### Train


In [5]:
from train_tools import train_offline

train_offline(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=1,
    G=G,
    F=F,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    device=device,
    early_stopping=True,
    patience=5,
    verbose=True,
    delta=0.001,
    path=CHECKPOINT_DIR,
)

Epoch 1:   0%|          | 0/3497 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/875 [00:00<?, ?it/s]

Epoch 0 - train_loss: 0.3456 - train_acc: 0.8484 - train_f1: 0.6582 - test_loss: 0.3074 - test_acc: 0.8731 - test_f1: 0.7469
Validation loss decreased (inf --> 0.307404), f1 score increased (-inf --> 0.746873).  Saving model ...


Epoch 2:   0%|          | 0/3497 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 314.00 MiB (GPU 0; 6.00 GiB total capacity; 5.12 GiB already allocated; 0 bytes free; 5.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF